<a href="https://colab.research.google.com/github/gbrao018/eva4/blob/master/Assignment2_Ganji.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 4, 3, padding=1) #input -? Output? Local RF=(3*3), input:(28*28*1),out=(28*28*8), RF=3*3 , input channels=1, output channels=32
        self.batch1=nn.BatchNorm2d(num_features=4)
        self.conv2 = nn.Conv2d(4, 8, 3, padding=1) #in=(28*28*32), out=(28*28,64),RF=5*5, input channels=8,output channels=16. Earlier 32 and current 64  kernels are working on same pixel only. RF not changing. 
        self.batch2=nn.BatchNorm2d(num_features=8)
        self.pool1 = nn.MaxPool2d(2, 2) # intput=(28*28,64), output=(14*14*64), Global RF (10*10). 
        self.conv3 = nn.Conv2d(8, 16, 3, padding=1) # input=(14*14*16), output=(14*14*32),RF=(12*12)
        self.batch3=nn.BatchNorm2d(num_features=16)
        self.conv4 = nn.Conv2d(16, 32, 3, padding=1) #input=(14*14*32) , output=(14*14*32),RF=(14*14)
        self.batch4=nn.BatchNorm2d(num_features=32)
        self.pool2 = nn.MaxPool2d(2, 2) # input=(14*14*32), output=(7*7*32),RF=(28*28)
        self.conv5 = nn.Conv2d(32, 32, 3) # input=(7*7*32), output=(5*5*32), no padding here,RF=(30*30)
        self.batch5=nn.BatchNorm2d(num_features=32)
        self.conv6 = nn.Conv2d(32, 32, 3) # input=(5*5*32), output=(3*3*32), no padding here,RF=(30*30)
        self.batch6=nn.BatchNorm2d(num_features=32)
        self.conv7 = nn.Conv2d(32, 10, 3) # input=(5*5*32), output=(3*3*32), no padding here,RF=(30*30)
        #self.batch7=nn.BatchNorm2d(num_features=10)
        #self.conv8 = nn.Conv2d(10, 10, 3) # input=(3*3*32), output=(3*3*32), no padding here,RF=(30*30)
        

    def forward(self, x):
        x = self.pool1(F.dropout(self.batch2(F.relu(self.conv2(F.dropout(self.batch1(F.relu(self.conv1(x)))))))))
        x = self.pool2(F.dropout(self.batch4(F.relu(self.conv4(F.dropout(self.batch3(F.relu(self.conv3(x)))))))))
        x = self.batch6(F.relu(self.conv6(self.batch5(F.relu(self.conv5(x))))))
        x = self.conv7(x)
        x = x.view(-1, 10)
        y=F.log_softmax(x)
        return F.log_softmax(x)

In [166]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 4, 28, 28]              40
       BatchNorm2d-2            [-1, 4, 28, 28]               8
            Conv2d-3            [-1, 8, 28, 28]             296
       BatchNorm2d-4            [-1, 8, 28, 28]              16
         MaxPool2d-5            [-1, 8, 14, 14]               0
            Conv2d-6           [-1, 16, 14, 14]           1,168
       BatchNorm2d-7           [-1, 16, 14, 14]              32
            Conv2d-8           [-1, 32, 14, 14]           4,640
       BatchNorm2d-9           [-1, 32, 14, 14]              64
        MaxPool2d-10             [-1, 32, 7, 7]               0
           Conv2d-11             [-1, 32, 5, 5]           9,248
      BatchNorm2d-12             [-1, 32, 5, 5]              64
           Conv2d-13             [-1, 32, 3, 3]           9,248
      BatchNorm2d-14             [-1, 3

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


# New Section

# New Section

In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [164]:
#1. increased the epochs to 5, got 90% accuracy. Increasing further did not bring change
#2. no of params after
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)







loss=0.10277213901281357 batch_id=225:  48%|████▊     | 224/469 [00:07<00:07, 30.92it/s]





loss=0.1286022961139679 batch_id=226:  48%|████▊     | 224/469 [00:07<00:07, 30.92it/s] 





loss=0.04671086370944977 batch_id=227:  48%|████▊     | 224/469 [00:07<00:07, 30.92it/s]





loss=0.04671086370944977 batch_id=227:  49%|████▊     | 228/469 [00:07<00:07, 32.31it/s]





loss=0.0995173305273056 batch_id=228:  49%|████▊     | 228/469 [00:07<00:07, 32.31it/s] 





loss=0.059630658477544785 batch_id=229:  49%|████▊     | 228/469 [00:07<00:07, 32.31it/s]





loss=0.0636897087097168 batch_id=230:  49%|████▊     | 228/469 [00:07<00:07, 32.31it/s]  





loss=0.08553381264209747 batch_id=231:  49%|████▊     | 228/469 [00:07<00:07, 32.31it/s]





loss=0.08553381264209747 batch_id=231:  49%|████▉     | 232/469 [00:07<00:07, 32.78it/s]





loss=0.06439181417226791 batch_id=232:  49%|████▉     | 232/469 [00:07<00:07, 32.78it/s]





loss=0.07213117182254791 batch_id=233:  49%|████▉   


Test set: Average loss: 0.0673, Accuracy: 9795/10000 (98%)









loss=0.03065650537610054 batch_id=0:   0%|          | 0/469 [00:00<?, ?it/s]





loss=0.011801112443208694 batch_id=1:   0%|          | 0/469 [00:00<?, ?it/s]





loss=0.011801112443208694 batch_id=1:   0%|          | 2/469 [00:00<00:27, 16.74it/s]





loss=0.0489937849342823 batch_id=2:   0%|          | 2/469 [00:00<00:27, 16.74it/s]  





loss=0.21925729513168335 batch_id=3:   0%|          | 2/469 [00:00<00:27, 16.74it/s]





loss=0.07281465828418732 batch_id=4:   0%|          | 2/469 [00:00<00:27, 16.74it/s]





loss=0.04897552356123924 batch_id=5:   0%|          | 2/469 [00:00<00:27, 16.74it/s]





loss=0.04897552356123924 batch_id=5:   1%|▏         | 6/469 [00:00<00:23, 20.05it/s]





loss=0.12125012278556824 batch_id=6:   1%|▏         | 6/469 [00:00<00:23, 20.05it/s]





loss=0.05262143537402153 batch_id=7:   1%|▏         | 6/469 [00:00<00:23, 20.05it/s]





loss=0.05592658370733261 batch_id=8:   1%|▏         | 6/469 [00:00<00:23, 20.05it/s]





loss=0.0777345001


Test set: Average loss: 0.0696, Accuracy: 9764/10000 (98%)









loss=0.07303167134523392 batch_id=0:   0%|          | 0/469 [00:00<?, ?it/s]





loss=0.030504688620567322 batch_id=1:   0%|          | 0/469 [00:00<?, ?it/s]





loss=0.030504688620567322 batch_id=1:   0%|          | 2/469 [00:00<00:24, 19.15it/s]





loss=0.03425682336091995 batch_id=2:   0%|          | 2/469 [00:00<00:24, 19.15it/s] 





loss=0.074874646961689 batch_id=3:   0%|          | 2/469 [00:00<00:24, 19.15it/s]  





loss=0.08336248993873596 batch_id=4:   0%|          | 2/469 [00:00<00:24, 19.15it/s]





loss=0.05321185290813446 batch_id=5:   0%|          | 2/469 [00:00<00:24, 19.15it/s]





loss=0.05321185290813446 batch_id=5:   1%|▏         | 6/469 [00:00<00:20, 22.33it/s]





loss=0.1172977015376091 batch_id=6:   1%|▏         | 6/469 [00:00<00:20, 22.33it/s] 





loss=0.053048186004161835 batch_id=7:   1%|▏         | 6/469 [00:00<00:20, 22.33it/s]





loss=0.24764388799667358 batch_id=8:   1%|▏         | 6/469 [00:00<00:20, 22.33it/s] 





loss=0.11248784

...


Test set: Average loss: 0.0563, Accuracy: 9816/10000 (98%)









loss=0.0271894708275795 batch_id=0:   0%|          | 0/469 [00:00<?, ?it/s]





loss=0.05703439190983772 batch_id=1:   0%|          | 0/469 [00:00<?, ?it/s]





loss=0.05703439190983772 batch_id=1:   0%|          | 2/469 [00:00<00:29, 16.07it/s]





loss=0.1536390632390976 batch_id=2:   0%|          | 2/469 [00:00<00:29, 16.07it/s] 





loss=0.0451185405254364 batch_id=3:   0%|          | 2/469 [00:00<00:29, 16.07it/s]





loss=0.12425325810909271 batch_id=4:   0%|          | 2/469 [00:00<00:29, 16.07it/s]





loss=0.010624844580888748 batch_id=5:   0%|          | 2/469 [00:00<00:29, 16.07it/s]





loss=0.010624844580888748 batch_id=5:   1%|▏         | 6/469 [00:00<00:23, 19.43it/s]





loss=0.11477825790643692 batch_id=6:   1%|▏         | 6/469 [00:00<00:23, 19.43it/s] 





loss=0.12380626052618027 batch_id=7:   1%|▏         | 6/469 [00:00<00:23, 19.43it/s]





loss=0.1206914409995079 batch_id=8:   1%|▏         | 6/469 [00:00<00:23, 19.43it/s] 





loss=0.036080390214


Test set: Average loss: 0.0602, Accuracy: 9799/10000 (98%)









loss=0.06541638821363449 batch_id=0:   0%|          | 0/469 [00:00<?, ?it/s]





loss=0.024304956197738647 batch_id=1:   0%|          | 0/469 [00:00<?, ?it/s]





loss=0.024304956197738647 batch_id=1:   0%|          | 2/469 [00:00<00:26, 17.93it/s]





loss=0.059704601764678955 batch_id=2:   0%|          | 2/469 [00:00<00:26, 17.93it/s]





loss=0.03424786403775215 batch_id=3:   0%|          | 2/469 [00:00<00:26, 17.93it/s] 





loss=0.07130444049835205 batch_id=4:   0%|          | 2/469 [00:00<00:26, 17.93it/s]





loss=0.04813968390226364 batch_id=5:   0%|          | 2/469 [00:00<00:26, 17.93it/s]





loss=0.04813968390226364 batch_id=5:   1%|▏         | 6/469 [00:00<00:21, 21.26it/s]





loss=0.031890563666820526 batch_id=6:   1%|▏         | 6/469 [00:00<00:21, 21.26it/s]





loss=0.10500455647706985 batch_id=7:   1%|▏         | 6/469 [00:00<00:21, 21.26it/s] 





loss=0.023119863122701645 batch_id=8:   1%|▏         | 6/469 [00:00<00:21, 21.26it/s]





loss=0.027929


Test set: Average loss: 0.0567, Accuracy: 9812/10000 (98%)

